In [32]:
# %pip install -r ../requirements.txt
# %pip install --quiet --upgrade  langchain langchain-community langchainhub gpt4all chromadb bs4 torch transformers
# !pip freeze >> ../requirements.txt

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings, GPT4AllEmbeddings, BedrockEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import LlamaCpp, Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnablePick

from chromadb.errors import InvalidDimensionException

In [33]:
import os

from langchain_community.document_loaders import TextLoader

BASE_DIR = '/home/raj/nlp/cmu-rag/data/documents/combined_txt_files_length_normalized/'
# relevant_files = ['l']

docs = []
for file in os.listdir(BASE_DIR):
    # print(file)
    if "schedule" in file:
        continue
    print("Using file :", file)
    loader = TextLoader(BASE_DIR + file)
    docs.extend(loader.load())
    break

print("Splitting Text")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)
print("Splitting Text Done")

# try:
#     print("Creating Chroma")   
#     vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings())
# except InvalidDimensionException:
#     print("Deleting Chroma")
#     Chroma().delete_collection()
#     print("Creating Chroma afresh")
#     vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings())
# print("Done Creating Chroma")

Using file : Buggy News_part_0
Splitting Text
Splitting Text Done


In [34]:
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = PromptTemplate.from_template(
    "Answer questions on CMU from these documents"
)

llm = Ollama(model="llama2")

rag_prompt_llama = hub.pull("rlm/rag-prompt-llama")
prompt = rag_prompt_llama
# print(prompt.messages)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


NameError: name 'vectorstore' is not defined

In [27]:
questions = ['Whom should I contact for additional info on the MCDS program?', 'How many semesters does the MS AI program generally consist of ?']

ex = qa_chain.invoke(questions)

NameError: name 'qa_chain' is not defined

In [50]:
print(ex)
type(ex)

For the first question, "Whom should I contact for additional info on the MCDS program?", you can reach out to the McMaster University Graduate Studies office or visit their website for more information. They have a dedicated team for handling inquiries and can provide you with the most up-to-date and accurate information about the program.

For the second question, "How many semesters does the MS AI program generally consist of?", the answer is generally two to three semesters, depending on the student's academic background and the course load they take on. It's best to consult with the program advisor or check the university's website for the most accurate information.


str

In [ ]:
db = Chroma(persist_directory="./chroma_db", embedding_function=OllamaEmbeddings())

In [51]:
from langchain_nomic import NomicEmbeddings
import ollama

nomic_key = 'nk-VBwHqnKuow04jykyGp4JEBDUmmPInl6nDV2dQNEMVto'
!nomic login $nomic_key
# nomic = NomicEmbeddings(key = nomic_key, model = 'nomic-embed-text-v1.5')

from nomic import embed

output = embed.text(
    texts=['Nomic Embedding API', '#keepAIOpen'],
    model='nomic-embed-text-v1.5',
    task_type='search_document',
    dimensionality=256,
)

print(output)


vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings(model = 'nomic-embed-text'))

# output = embed.text(
#     texts=['Nomic Embedding API', '#keepAIOpen'],
#     model='nomic-embed-text-v1',
#     task_type='search_document'
# )

# print(output)



{'embeddings': [[0.014526367, 0.024505615, -0.21765137, -0.095214844, 0.079589844, -0.06542969, -0.109313965, -0.11010742, -0.10296631, -0.07763672, -0.08239746, 0.008018494, 0.08996582, 0.0076293945, -0.05633545, -0.031280518, -0.015792847, -0.07647705, -0.031143188, 0.0791626, -0.03036499, -0.06817627, -0.012901306, -0.016494751, 0.048706055, -0.01209259, 0.20080566, -0.028549194, -0.050323486, -0.007762909, -0.022979736, -0.031311035, 0.017578125, -0.10449219, 0.03933716, -0.028945923, -0.03692627, 0.09234619, -0.015220642, 0.06112671, -0.006072998, -0.040618896, -0.0076675415, -0.0016126633, -0.0047798157, 0.0028152466, 0.10449219, -0.050720215, 0.04724121, -0.15734863, 0.046661377, -0.08001709, -0.015625, -0.034851074, 0.10900879, -0.045806885, -0.03527832, 0.125, 0.00806427, 0.025100708, 0.09576416, 0.119018555, -0.032714844, 0.083496094, 0.08496094, 0.006298065, -0.13378906, 0.15527344, 0.028335571, -0.036346436, 0.049835205, 0.073913574, 0.027130127, -0.042144775, -0.015525818,

KeyboardInterrupt: 